In [12]:
from docx import Document
import re
from collections import defaultdict
import spacy
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Charger le modèle NLP pour le résumé automatique
nlp = spacy.load("en_core_web_sm")

def extract_themes_summaries(doc_path):
    doc = Document(doc_path)
    
    # Vérification du nombre de paragraphes
    if len(doc.paragraphs) == 0:
        print("Aucun paragraphe détecté dans le document.")
        return {}
    
    themes = defaultdict(list)
    current_theme = "Autre"  # Thème par défaut
    
    for para in doc.paragraphs:
        text = para.text.strip()
        
        # Affichage des styles pour diagnostic
        print(f"Texte: {text[:50]}... | Style: {para.style.name}")
        
        # Détection améliorée des thématiques (inclut titres sans styles spécifiques)
        if (
            para.style.name in ['Heading 1', 'Heading 2', 'Heading 3'] or
            text.isupper() or
            re.match(r'^(\d+\.\s+|[A-Z][a-z]+\s+[A-Z][a-z]+)', text) or  # Détection de numérotation et titres courants
            len(text.split()) < 6  # Texte court = probablement un titre
        ):
            current_theme = text
            themes[current_theme] = []
            continue
        
        # Ajout du texte sous la thématique courante
        if current_theme in themes:
            themes[current_theme].append(text)
    
    # Vérification des thématiques extraites
    print("Thématiques détectées:", themes.keys())
    
    # Génération d'un résumé optimisé par NLP et TextRank
    summaries = {}
    for theme, texts in themes.items():
        full_text = " ".join(texts)
        if not full_text.strip():
            summaries[theme] = "Aucune information disponible."
            continue
        
        doc_nlp = nlp(full_text)
        sentences = [sent.text for sent in doc_nlp.sents]
        
        if len(sentences) > 3:
            # Utilisation de TF-IDF pour identifier les phrases les plus informatives
            vectorizer = TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform(sentences)
            sentence_scores = np.array(tfidf_matrix.sum(axis=1)).flatten()
            top_sentence_indices = sentence_scores.argsort()[-3:][::-1]
            summary_sentences = [sentences[i] for i in sorted(top_sentence_indices)]
            summaries[theme] = " ".join(summary_sentences)
        else:
            summaries[theme] = full_text if full_text else "Aucune information disponible."
    
    return summaries

# Exemple d'utilisation
doc_path = "../data/projet_1/PU_P01_AAP01.docx"  # Remplace par ton fichier
theme_summaries = extract_themes_summaries(doc_path)

# Affichage formaté des résultats
for theme, summary in theme_summaries.items():
    print(f"\n### {theme} ###")
    print(f"Résumé: {summary}\n")

Texte: * Please fill it out briefly with a total length l... | Style: Normal
Texte: ... | Style: Normal
Texte: For Standard Grant... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: Project objective　(Explain which GBF goal project ... | Style: Normal
Texte: Note: Please specify which target(s) your project ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: 2.  Project implementation plan... | Style: Normal
Texte: (Please describe the projects by year and items.)... | Style: Normal
Texte: Note; For project, please describe not only fiscal... | Style: Normal
Texte: ... | Style: Normal
Texte: Applying FY... | Style: List Paragraph
Texte: FY(s) before applying FY... | Style: List Paragraph
Texte: FY(s) after applying FY... | Style: List Paragraph
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: 3．Expected concrete

In [1]:
import requests

response = requests.post(
    "http://127.0.0.1:11435/api/generate",
    json={"model": "mistral", "prompt": "Bonjour, comment vas-tu ?", "stream": False},
)

print(response.json())


{'error': "model 'mistral' not found"}


In [14]:
from docx import Document
import re
from collections import defaultdict
import spacy
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Charger le modèle NLP pour le résumé automatique
nlp = spacy.load("en_core_web_sm")

def is_valid_theme(text):
    """ Vérifie si un texte est un titre valide """
    invalid_starts = ["Note", "・", "Attach", "Please", "1000 caractères restants", "Ce champ est requis"]
    if len(text.split()) < 3 or any(text.startswith(prefix) for prefix in invalid_starts):
        return False
    return True

def extract_themes_summaries(doc_paths):
    all_summaries = {}
    
    for doc_path in doc_paths:
        doc = Document(doc_path)
        
        # Vérification du nombre de paragraphes
        if len(doc.paragraphs) == 0:
            print(f"Aucun paragraphe détecté dans {doc_path}.")
            continue
        
        themes = defaultdict(list)
        current_theme = "Autre"  # Thème par défaut
        
        for para in doc.paragraphs:
            text = para.text.strip()
            
            # Affichage des styles pour diagnostic
            print(f"Texte: {text[:50]}... | Style: {para.style.name}")
            
            # Détection améliorée des thématiques
            if (
                para.style.name in ['Heading 1', 'Heading 2', 'Heading 3'] or
                text.isupper() or
                re.match(r'^(\d+\.\s+|[A-Z][a-z]+\s+[A-Z][a-z]+)', text) or  # Détection numérotation et titres
                len(text.split()) < 6  # Texte court = probablement un titre
            ) and is_valid_theme(text):
                current_theme = text
                themes[current_theme] = []
                continue
            
            # Ajout du texte sous la thématique courante
            if current_theme in themes:
                themes[current_theme].append(text)
        
        # Vérification des thématiques extraites
        print(f"Thématiques détectées dans {doc_path}: {themes.keys()}")
        
        # Génération d'un résumé optimisé par NLP et TextRank
        summaries = {}
        for theme, texts in themes.items():
            full_text = " ".join(texts)
            if not full_text.strip():
                summaries[theme] = "Aucune information disponible."
                continue
            
            doc_nlp = nlp(full_text)
            sentences = [sent.text for sent in doc_nlp.sents]
            
            if len(sentences) > 3:
                # Utilisation de TF-IDF pour identifier les phrases les plus informatives
                vectorizer = TfidfVectorizer()
                tfidf_matrix = vectorizer.fit_transform(sentences)
                sentence_scores = np.array(tfidf_matrix.sum(axis=1)).flatten()
                top_sentence_indices = sentence_scores.argsort()[-3:][::-1]
                summary_sentences = [sentences[i] for i in sorted(top_sentence_indices)]
                summaries[theme] = " ".join(summary_sentences)
            else:
                # Sélectionner la phrase la plus longue si le résumé est vide
                longest_sentence = max(sentences, key=len, default="Aucune information disponible.")
                summaries[theme] = longest_sentence
        
        all_summaries[doc_path] = summaries
    
    return all_summaries
# Exemple d'utilisation
doc_paths = ["../data/projet_1/PU_P01_AAP01.docx", "../data/projet_1/PU_P01_AAP02.docx", "../data/projet_1/PU_P01_AAP03.docx"]
theme_summaries = extract_themes_summaries(doc_paths)

# Affichage formaté des résultats
for doc, summaries in theme_summaries.items():
    print(f"\n===== Résumé pour {doc} =====")
    for theme, summary in summaries.items():
        print(f"\n### {theme} ###")
        print(f"Résumé: {summary}\n")

Texte: * Please fill it out briefly with a total length l... | Style: Normal
Texte: ... | Style: Normal
Texte: For Standard Grant... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: Project objective　(Explain which GBF goal project ... | Style: Normal
Texte: Note: Please specify which target(s) your project ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: 2.  Project implementation plan... | Style: Normal
Texte: (Please describe the projects by year and items.)... | Style: Normal
Texte: Note; For project, please describe not only fiscal... | Style: Normal
Texte: ... | Style: Normal
Texte: Applying FY... | Style: List Paragraph
Texte: FY(s) before applying FY... | Style: List Paragraph
Texte: FY(s) after applying FY... | Style: List Paragraph
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: 3．Expected concrete

In [2]:
import requests
from docx import Document
import re
from collections import defaultdict

OLLAMA_HOST = "http://127.0.0.1:11435/api/generate"

def is_valid_theme(text):
    """ Vérifie si un texte est un titre valide """
    invalid_starts = ["Note", "・", "Attach", "Please", "1000 caractères restants", "Ce champ est requis"]
    return len(text.split()) >= 3 and not any(text.startswith(prefix) for prefix in invalid_starts)

def ask_mixtral(prompt):
    """ Envoie une requête au modèle Mixtral pour classifier et résumer le texte """
    response = requests.post(
        OLLAMA_HOST,
        json={"model": "mixtral", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "Erreur: pas de réponse")

def extract_themes_summaries(doc_paths):
    all_summaries = {}

    for doc_path in doc_paths:
        doc = Document(doc_path)
        if len(doc.paragraphs) == 0:
            print(f"Aucun paragraphe détecté dans {doc_path}.")
            continue
        
        themes = defaultdict(list)
        current_theme = "Autre"  # Thème par défaut

        for para in doc.paragraphs:
            text = para.text.strip()
            if (
                para.style.name in ['Heading 1', 'Heading 2', 'Heading 3'] or
                text.isupper() or
                re.match(r'^(\d+\.\s+|[A-Z][a-z]+\s+[A-Z][a-z]+)', text) or
                len(text.split()) < 6
            ) and is_valid_theme(text):
                current_theme = text
                themes[current_theme] = []
                continue

            if current_theme in themes:
                themes[current_theme].append(text)

        print(f"Thématiques détectées dans {doc_path}: {themes.keys()}")

        summaries = {}
        for theme, texts in themes.items():
            full_text = " ".join(texts)
            if not full_text.strip():
                summaries[theme] = "Aucune information disponible."
                continue

            prompt = f"Catégorise et résume la section suivante d'un document d'appel à projet:\n\n{full_text}"
            mixtral_response = ask_mixtral(prompt)
            summaries[theme] = mixtral_response
        
        all_summaries[doc_path] = summaries

    return all_summaries

# Exemple d'utilisation avec tes fichiers
doc_paths = ["../data/projet_1/PU_P01_AAP01.docx", "../data/projet_1/PU_P01_AAP02.docx", "../data/projet_1/PU_P01_AAP03.docx"]
theme_summaries = extract_themes_summaries(doc_paths)

# Affichage formaté des résultats
for doc, summaries in theme_summaries.items():
    print(f"\n===== Résumé pour {doc} =====")
    for theme, summary in summaries.items():
        print(f"\n### {theme} ###")
        print(f"Résumé et Classification: {summary}\n")


Thématiques détectées dans ../data/projet_1/PU_P01_AAP01.docx: dict_keys(['For Standard Grant', '2.  Project implementation plan', 'FY(s) before applying FY', 'FY(s) after applying FY', '3．Expected concrete activity results', '（１）\tApplying FY', '（２）\tFY(s) before applying FY', '（３）\tFY(s) after applying FY', '4.  Activity schedule', '4. Activity schedule', '5. Income and Expenditure Budget plan', '6. The name and contact information of the experts outside of applicant’s who will provive advice and guidance for the project implementation .', '7. Local approvals (if you need local government approvals or agreements with local residents, please describe the contents)', '8. Partners (if you partner with a local NGO or other international organization on the projects, please list the names of the organizations)', '9. Japanese Introducer (Name and contact information)', 'Form of the organization'])
Thématiques détectées dans ../data/projet_1/PU_P01_AAP02.docx: dict_keys(['Description du pro

In [4]:
import requests
from docx import Document

OLLAMA_HOST = "http://127.0.0.1:11435/api/generate"

def extract_text_by_sections(doc_path):
    """ Extrait le texte par sections d'un document Word """
    doc = Document(doc_path)
    sections = {}
    current_section = "Introduction"  # Nom générique si aucun titre n'est trouvé

    for para in doc.paragraphs:
        text = para.text.strip()
        if text and len(text.split()) < 10:  # Supposons que les titres sont courts
            current_section = text
            sections[current_section] = []
        elif text:
            sections[current_section].append(text)

    return {sec: " ".join(content) for sec, content in sections.items()}

def ask_mixtral(prompt):
    """ Envoie une requête à Mixtral pour analyse """
    response = requests.post(
        OLLAMA_HOST,
        json={"model": "mixtral", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "Erreur: pas de réponse")

# Charger le PU_P01_PP pour connaître les sections requises
pp_sections = extract_text_by_sections("../data/projet_1/PU_P01_PP.docx")

# Charger et comparer les AAP
aap_paths = ["../data/projet_1/PU_P01_AAP01.docx", "../data/projet_1/PU_P01_AAP02.docx", "../data/projet_1/PU_P01_AAP03.docx"]
results = {}

for aap_path in aap_paths:
    aap_sections = extract_text_by_sections(aap_path)

    # Construire la requête pour Mixtral
    prompt = f"""
    Voici les sections attendues dans un appel à projets valide:
    {list(pp_sections.keys())}

    Voici les sections trouvées dans le document {aap_path}:
    {list(aap_sections.keys())}

    Vérifie si toutes les sections requises sont présentes.
    - Si elles sont toutes présentes, réponds simplement : "✅ Toutes les informations requises sont présentes."
    - Sinon, liste les sections manquantes en format simple : "❌ Manque les sections : [Liste]".
    """

    mixtral_response = ask_mixtral(prompt)
    results[aap_path] = mixtral_response

# Affichage des résultats
for doc, result in results.items():
    print(f"\n===== Vérification pour {doc} =====")
    print(result)


KeyError: 'Introduction'

In [5]:
import requests
from docx import Document

OLLAMA_HOST = "http://127.0.0.1:11435/api/generate"

def extract_text_by_sections(doc_path):
    """ Extrait le texte par sections d'un document Word en structurant les parties détectées """
    doc = Document(doc_path)
    sections = {}
    current_section = "Section Générique"  # Ajout d'une section par défaut

    for para in doc.paragraphs:
        text = para.text.strip()
        if text:
            # Détecter les titres comme les sections (basé sur la longueur et la mise en forme)
            if len(text.split()) < 10 and text[-1] not in ".!?":
                current_section = text
                sections[current_section] = []
            else:
                if current_section not in sections:
                    sections[current_section] = []  # Évite les erreurs KeyError
                sections[current_section].append(text)

    return {sec: " ".join(content) for sec, content in sections.items() if content}

def ask_mixtral(prompt):
    """ Envoie une requête à Mixtral pour analyse """
    response = requests.post(
        OLLAMA_HOST,
        json={"model": "mixtral", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "Erreur: pas de réponse")

# Charger le PU_P01_PP pour connaître les sections requises
pp_sections = extract_text_by_sections("../data/projet_1/PU_P01_PP.docx")

# Charger et comparer les AAP
aap_paths = ["../data/projet_1/PU_P01_AAP01.docx", "../data/projet_1/PU_P01_AAP02.docx", "../data/projet_1/PU_P01_AAP03.docx"]
results = {}

for aap_path in aap_paths:
    aap_sections = extract_text_by_sections(aap_path)

    # Construire la requête pour Mixtral
    prompt = f"""
    Voici les sections attendues dans un appel à projets valide:
    {list(pp_sections.keys())}

    Voici les sections trouvées dans le document {aap_path}:
    {list(aap_sections.keys())}

    Vérifie si toutes les sections requises sont présentes.
    - Si elles sont toutes présentes, réponds simplement : "✅ Toutes les informations requises sont présentes."
    - Sinon, liste les sections manquantes en format simple : "❌ Manque les sections : [Liste]".
    """

    mixtral_response = ask_mixtral(prompt)
    results[aap_path] = mixtral_response

# Affichage des résultats
for doc, result in results.items():
    print(f"\n===== Vérification pour {doc} =====")
    print(result)



===== Vérification pour ../data/projet_1/PU_P01_AAP01.docx =====
❌ Manque les sections : ['Brief project description', 'Name of the organization', 'Address', 'Restoring mangrove ecosystems, especially in the Mahakam Delta', 'Some activities conducted by POKJA Pesisir', 'Geographic and socio-economic context', 'Environmental context', 'Table 1. Critical Criteria of Mahakam Delta Mangrove', 'Biodiversity issues', 'Institutional Context', 'The Movement of Indonesian New Capital', 'Aquaculture industry', 'Demography', 'Other issues', 'Strategy & theory of change', 'Lack of alternative sustainable livelihood in coastal area;', 'Environmental awareness;', 'Beneficiaries', 'Teachers (primary school teachers)', 'Village officials', 'Location', '1.1.1. -Workshop on Environmental Conservation for Primary School Teachers', '1.2.1 -Encouraging/assisting schools towards adiwiyata in Mahakam Delta', '1.3.1. Waste Management Campaign', 'Target: 100 persons local community involved', 'Health approach

In [7]:
import requests
from docx import Document

OLLAMA_HOST = "http://127.0.0.1:11435/api/generate"

def extract_text_by_sections(doc_path):
    """ Extrait le texte des sections d'un document Word """
    doc = Document(doc_path)
    sections = {}
    current_section = "Section Générique"

    for para in doc.paragraphs:
        text = para.text.strip()
        if text:
            if len(text.split()) < 10 and text[-1] not in ".!?":
                current_section = text
                sections[current_section] = []
            else:
                if current_section not in sections:
                    sections[current_section] = []
                sections[current_section].append(text)

    return {sec: " ".join(content) for sec, content in sections.items() if content}

def ask_mixtral(prompt):
    """ Envoie une requête à Mixtral pour analyse et compréhension du texte """
    response = requests.post(
        OLLAMA_HOST,
        json={"model": "mixtral", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "Erreur: pas de réponse")

# Charger les sections attendues du PU_P01_PP
pp_sections = extract_text_by_sections("../data/projet_1/PU_P01_PP.docx")

# Charger et comparer les AAP
aap_paths = ["../data/projet_1/PU_P01_AAP01.docx", "../data/projet_1/PU_P01_AAP02.docx", "../data/projet_1/PU_P01_AAP03.docx"]
results = {}

for aap_path in aap_paths:
    aap_sections = extract_text_by_sections(aap_path)

    # Construire une requête LLM plus avancée
    prompt = f"""
    Voici les sections attendues dans un appel à projet valide (PU_P01_PP) :
    {list(pp_sections.keys())}

    Voici les sections trouvées dans {aap_path} :
    {list(aap_sections.keys())}

    **Tâche** :
    - Vérifie si chaque section attendue est bien couverte dans le document.
    - Même si le titre diffère, vérifie si le contenu correspondant est présent sous une autre section.
    - Si tout est bien couvert, réponds : ✅ Toutes les informations requises sont présentes.
    - Si des parties sont absentes ou incomplètes, liste clairement les sections manquantes : "❌ Manque les sections : [Liste]".
    """

    mixtral_response = ask_mixtral(prompt)
    results[aap_path] = mixtral_response

# Affichage des résultats
for doc, result in results.items():
    print(f"\n===== Vérification pour {doc} =====")
    print(result)



===== Vérification pour ../data/projet_1/PU_P01_AAP01.docx =====
❌ Manque les sections : ['Brief project description', 'Name of the organization: Pokja Pesisir', 'Website: http://pokjapesisir.id/', 'Some activities conducted by POKJA Pesisir:', 'Geographic and socio-economic context', 'Environmental context', 'Table 1. Critical Criteria of Mahakam Delta Mangrove', 'Biodiversity issues', 'Institutional Context', 'The Movement of Indonesian New Capital', 'Aquaculture industry', 'Demography', 'Other issues', 'Strategy & theory of change', 'Lack of alternative sustainable livelihood in coastal area;', 'Environmental awareness;', 'Beneficiaries', 'Teachers (primary school teachers)', 'Village officials', 'Location', 'CAPACITY BUILDING PLAN', 'Cross-cutting approaches', 'Knowledge management', 'Capitalization', 'External communication in Indonesia', 'External communication in France', 'Sustainability, scaling-up and/or exit strategy', 'Sustainability and Exit Strategy (Scale Up) of the proj

In [ ]:
import requests
from docx import Document

OLLAMA_HOST = "http://127.0.0.1:11435/api/generate"

def extract_text_by_sections(doc_path):
    """ Extrait le texte des sections d'un document Word """
    doc = Document(doc_path)
    sections = {}
    current_section = "Section Générique"

    for para in doc.paragraphs:
        text = para.text.strip()
        if text:
            if len(text.split()) < 10 and text[-1] not in ".!?":
                current_section = text
                sections[current_section] = []
            else:
                if current_section not in sections:
                    sections[current_section] = []
                sections[current_section].append(text)

    return {sec: " ".join(content) for sec, content in sections.items() if content}

def ask_mixtral(prompt):
    """ Envoie une requête à Mixtral pour analyse et compréhension du texte """
    response = requests.post(
        OLLAMA_HOST,
        json={"model": "mixtral", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "Erreur: pas de réponse")

# Charger les sections attendues du PU_P01_PP (document de référence)
pp_sections = extract_text_by_sections("../data/projet_1/PU_P01_PP.docx")

# Charger et comparer les AAP
aap_paths = ["../data/projet_1/PU_P01_AAP01.docx", "../data/projet_1/PU_P01_AAP02.docx", "../data/projet_1/PU_P01_AAP03.docx"]
results = {}

for aap_path in aap_paths:
    aap_sections = extract_text_by_sections(aap_path)

    # Construire une requête avancée pour Mixtral
    prompt = f"""
    Nous devons vérifier si **les thématiques et contenus** du document de référence (PU_P01_PP) sont bien abordés dans {aap_path}.  
    Voici les **thématiques et contenus** attendus :
    {list(pp_sections.keys())}

    **Contenu du document analysé (AAP) :**
    {list(aap_sections.keys())}

    **Tâches :**
    1️⃣ Pour chaque thématique attendue, vérifie si elle est bien abordée dans l'AAP, même sous un autre titre.  
    2️⃣ Si une thématique est **totalement couverte**, marque-la ✅.  
    3️⃣ Si une thématique est **partiellement couverte**, marque-la ⚠️ et explique ce qui manque.  
    4️⃣ Si une thématique est totalement absente, marque-la ❌.  
    5️⃣ Donne une liste claire des thématiques **partiellement ou totalement absentes**.  

    **Réponse attendue :**
    - ✅ [Nom de la thématique] : Bien couverte.
    - ⚠️ [Nom de la thématique] : Partiellement couverte, éléments manquants : [détails].
    - ❌ [Nom de la thématique] : Absente.
    """

    mixtral_response = ask_mixtral(prompt)
    results[aap_path] = mixtral_response

# Affichage des résultats
for doc, result in results.items():
    print(f"\n===== Vérification thématique pour {doc} =====")
    print(result)
